PROCUREMENT FRAUD DETECTION - DATA PREPARATION


Author: Zeeshan

Date: January 2, 2026

Project: Hack4Delhi - Government Procurement Fraud Detection

Purpose: Clean and combine Assam + Himachal Pradesh tender data
for fraud pattern analysis

Data Sources:
- Assam eProcurement Portal (34,231 tenders)
- Himachal Pradesh Procurement (3,792 tenders)

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("🔧 DATA PREPARATION PIPELINE")
print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

🔧 DATA PREPARATION PIPELINE
Started at: 2026-01-02 00:31:48


In [3]:
# Load Assam tender records
# Note: This data is from public eProcurement portal
# Contains tender details but lacks internal payment records
assam_tenders = pd.read_csv('data/raw/assam_extracted/full/main.csv')
print(f"✓ Assam data loaded: {len(assam_tenders):,} records")

# Load Himachal Pradesh tender records  
# Similar structure but some column name differences
himachal_tenders = pd.read_csv('data/raw/himachal_extracted/full/main.csv')
print(f"✓ Himachal data loaded: {len(himachal_tenders):,} records")

# Quick inspection - what columns do we have?
print("\n📋 Assam columns available:")
print(assam_tenders.columns.tolist()[:10])  # Show first 10

print("\n📋 Himachal columns available:")
print(himachal_tenders.columns.tolist()[:10])


✓ Assam data loaded: 34,232 records
✓ Himachal data loaded: 3,791 records

📋 Assam columns available:
['_link', 'id', 'tag', 'date', 'ocid', 'Payment Mode', 'initiationType', 'fiscal_year', 'buyer_name', 'tender_id']

📋 Himachal columns available:
['_link', 'id', 'tag', 'date', 'ocid', 'initiationType', 'award_date', 'buyer_id', 'buyer_name', 'tender_id']


CLEANING STRATEGY :

1. Remove duplicates - Same tender_id appearing multiple times
2. Remove invalid amounts - Zero, negative, or null values
3. Remove missing dates - Can't do time analysis without dates
4. Keep only essential columns - Focus on fraud-relevant fields

Why these steps? Because fraud detection needs:
- Unique records (no double counting)
- Valid amounts (for price analysis)
- Proper dates (for timing patterns)

In [4]:
# ASSAM CLEANING
print("🧹 Cleaning Assam data...")

# Step 1: Remove duplicates based on tender ID
assam_initial = len(assam_tenders)
assam_tenders = assam_tenders.drop_duplicates(subset=['tender_id'], keep='first')
print(f"   Removed {assam_initial - len(assam_tenders)} duplicate tenders")

# Step 2: Remove invalid amounts
assam_tenders = assam_tenders[assam_tenders['tender_value_amount'].notna()]
assam_tenders['tender_value_amount'] = pd.to_numeric(assam_tenders['tender_value_amount'], errors='coerce')
assam_tenders = assam_tenders[assam_tenders['tender_value_amount'] > 0]
print(f"   Removed tenders with invalid amounts")

# Step 3: Remove missing dates
assam_tenders = assam_tenders[assam_tenders['date'].notna()]
print(f"   Final Assam records: {len(assam_tenders):,}")

# HIMACHAL CLEANING (similar logic)
print("\n🧹 Cleaning Himachal data...")

himachal_initial = len(himachal_tenders)
himachal_tenders = himachal_tenders.drop_duplicates(subset=['ocid'], keep='first')
print(f"   Removed {himachal_initial - len(himachal_tenders)} duplicate tenders")

himachal_tenders = himachal_tenders[himachal_tenders['tender_value_amount'].notna()]
himachal_tenders['tender_value_amount'] = pd.to_numeric(himachal_tenders['tender_value_amount'], errors='coerce')
himachal_tenders = himachal_tenders[himachal_tenders['tender_value_amount'] > 0]

himachal_tenders = himachal_tenders[himachal_tenders['tender_datePublished'].notna()]
print(f"   Final Himachal records: {len(himachal_tenders):,}")

🧹 Cleaning Assam data...
   Removed 0 duplicate tenders
   Removed tenders with invalid amounts
   Final Assam records: 26,205

🧹 Cleaning Himachal data...
   Removed 0 duplicate tenders
   Final Himachal records: 3,754


COLUMN STANDARDIZATION (My Design):

Problem: Both datasets have different column names for same data
Solution: Map them to common names I've chosen for this project

My naming convention:
- contract_id: Unique identifier
- pub_date: Publication/announcement date
- contract_amount: Tender value in rupees
- bidder_count: Number of companies that bid
- dept_name: Government department/buyer
- proc_method: Procurement type (open/limited/emergency)

In [5]:
# Create standardized Assam dataset
assam_standardized = pd.DataFrame({
    'contract_id': assam_tenders['tender_id'],
    'pub_date': assam_tenders['date'],
    'contract_amount': assam_tenders['tender_value_amount'],
    'bidder_count': assam_tenders['tender_numberOfTenderers'],
    'dept_name': assam_tenders['buyer_name'],
    'proc_method': assam_tenders['tender_procurementMethod'],
    'data_source': 'Assam'  # Track which state
})

# Create standardized Himachal dataset
himachal_standardized = pd.DataFrame({
    'contract_id': himachal_tenders['ocid'],
    'pub_date': himachal_tenders['tender_datePublished'],
    'contract_amount': himachal_tenders['tender_value_amount'],
    'bidder_count': himachal_tenders['tender_numberOfTenderers'],
    'dept_name': himachal_tenders['buyer_name'],
    'proc_method': himachal_tenders['tender_procurementMethod'],
    'data_source': 'Himachal Pradesh'
})

print("✓ Column standardization complete")
print(f"   Common columns: {assam_standardized.columns.tolist()}")

✓ Column standardization complete
   Common columns: ['contract_id', 'pub_date', 'contract_amount', 'bidder_count', 'dept_name', 'proc_method', 'data_source']


In [ ]:
# Combine both datasets into master dataset
master_data = pd.concat([assam_standardized, himachal_standardized], 
                        ignore_index=True)

print(f"\n✅ MASTER DATASET CREATED")
print(f"   Total records: {len(master_data):,}")
print(f"   Assam records: {len(assam_standardized):,} ({len(assam_standardized)/len(master_data)*100:.1f}%)")
print(f"   Himachal records: {len(himachal_standardized):,} ({len(himachal_standardized)/len(master_data)*100:.1f}%)")

# Save cleaned data
master_data.to_csv('data/processed/cleaned_master_data.csv', index=False)
print(f"\n💾 Saved to data/processed/cleaned_master_data.csv")

# Show sample
print("\n👀 Sample records:")
master_data.head(3)


✅ MASTER DATASET CREATED
   Total records: 29,959
   Assam records: 26,205 (87.5%)
   Himachal records: 3,754 (12.5%)

💾 Saved to: cleaned_master_data.csv

👀 Sample records:


,contract_id,pub_date,contract_amount,bidder_count,dept_name,proc_method,data_source
0,2016_DOT_946_1,2022-09-29,25132914.0,5.0,Department of Tourism,Open Tender,Assam
1,2016_DoWR_1302_1,2022-09-29,25349923.0,7.0,Department of Water Resources,Open Tender,Assam
2,2016_DoWR_1318_1,2022-09-29,7967277.0,5.0,Department of Water Resources,Open Tender,Assam
